# 컴퓨팅 대상 사용

스크립트를 Azure Machine Learning 실험으로 실행할 때는 실험 실행용 실행 컨텍스트를 정의해야 합니다. 실행 컨텍스트는 다음 항목으로 구성됩니다.

* 스크립트용 Python 환경. 스크립트에서 사용되는 모든 Python 패키지가 포함되어 있어야 합니다.
* 스크립트를 실행할 컴퓨팅 대상. 실험 실행이 시작되는 로컬 워크스테이션일 수도 있고 요청에 따라 프로비전되는 학습 클러스터 등의 원격 컴퓨팅 대상일 수도 있습니다.

이 Notebook에서는 실험용 *환경* 및 *컴퓨팅 대상*을 살펴봅니다.

## 작업 영역에 연결

이 Notebook의 작업을 시작하려면 먼저 작업 영역에 연결합니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 실험용 데이터 준비

이 Notebook에서는 당뇨병 환자의 세부 정보가 포함된 데이터 세트를 사용합니다. 아래 셀을 실행하여 이 데이터 세트를 만듭니다. 데이터 세트가 이미 있으면 코드가 기존 버전을 찾습니다.

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## 학습 스크립트 만들기

아래의 두 셀을 실행하여 다음 항목을 만듭니다.
1. 새 실험용 폴더
2. **scikit-learn**을 사용하여 모델을 학습시키고 **matplotlib**를 사용하여 ROC 곡선을 그리는 학습 스크립트 파일

In [ ]:
import os

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 스크립트 인수 가져오기
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# 정규화 하이퍼 매개 변수 설정
reg = args.reg_rate

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 로드(입력 데이터 세트로 전달됨)
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 로지스틱 회귀 모델 학습
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 곡선 그리기
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 대각선 50% 선 그리기
plt.plot([0, 1], [0, 1], 'k--')
# 모델의 FPR 및 TPR 그리기
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 환경 정의

Azure Machine Learning에서 Python 스크립트를 실험으로 실행하면 스크립트용 실행 컨텍스트를 정의하는 Conda 환경이 작성됩니다. Azure Machine Learning은 여러 공통 패키지가 포함된 기본 환경을 제공합니다. 이러한 패키지로는 실험 실행 사용에 필요한 라이브러리가 포함된 **azureml-defaults** 패키지, 그리고 **pandas**/**numpy** 등의 널리 사용되는 패키지 등이 있습니다.

**conda** 또는 **pip**를 사용하여 자체 환경을 정의하고 패키지를 추가할 수도 있습니다. 그러면 실험에서 필요한 모든 라이브러리에 액세스할 수 있습니다.

> **참고**: conda 종속성이 먼저 설치된 후 pip 종속성이 설치됩니다. pip 종속성을 설치하려면 **pip** 패키지가 필요하므로 conda 종속성에 이 패키지를 포함하는 것이 좋습니다. 패키지를 포함하지 않으면 Azure ML에서 자동으로 설치하지만 로그에 경고가 표시됩니다.

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# 실험용 Python 환경 만들기
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies

# 패키지 종속성 집합 만들기(필요에 따라 conda 또는 pip)
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib','pandas','pip'],
                                             pip_packages=['azureml-sdk','pyarrow'])

# 환경에 종속성 추가
diabetes_env.python.conda_dependencies = diabetes_packages

print(diabetes_env.name, 'defined.')

이제 환경을 사용하여 스크립트를 실험으로 실행할 수 있습니다.

다음 코드는 작성된 환경을 ScriptRunConfig에 할당하고 실험을 제출합니다. 실험이 실행되면 위젯과 **azureml_logs/60_control_log.txt** 출력 로그에서 실행 세부 정보를 관찰합니다. 그러면 conda 환경이 작성되고 있음을 확인할 수 있습니다.

> **참고**: 아래 코드는 스크립트 실행을 위한 **DockerConfiguration**을 만들며, 스크립트의 환경을 Docker 컨테이너에서 호스팅하기 위해 **user_docker** 특성을 **True**로 설정합니다. 이것은 기본 동작이므로 생략해도 됩니다. 하지만 여기에서는 확실히 설명하기 위해 포함시켰습니다.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                                environment=diabetes_env,
                                docker_runtime_config=DockerConfiguration(use_docker=True)) # Use docker to host environment

# 실험 제출
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

실험에서는 필요한 모든 패키지가 포함된 환경을 정상적으로 사용했습니다. 아래 코드를 실행하거나 Azure Machine Learning Studio를 통해 실험 실행의 출력과 메트릭을 확인할 수 있습니다. 예를 들어 **scikit-learn**을 사용하여 학습시킨 모델, **matplotlib**를 사용하여 생성된 ROC 차트 이미지 등을 확인할 수 있습니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 환경 등록

필요한 패키지가 포함된 환경을 정의한 후에는 작업 영역에 해당 환경을 등록할 수 있습니다.

In [ ]:
# 환경 등록
diabetes_env.register(workspace=ws)

환경은 처음 만들 때 할당한 이름(여기서는 *diabetes-experiment-env*)으로 등록됩니다.

환경을 등록하면 요구 사항이 같은 모든 스크립트에 해당 환경을 재사용할 수 있습니다. 여기서는 환경 재사용의 예를 확인하기 위해 다른 알고리즘을 사용하여 당뇨병 모델을 학습시키는 스크립트와 폴더를 만들어 보겠습니다.

In [ ]:
import os

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 스크립트 인수 가져오기
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 로드(입력 데이터 세트로 전달됨)
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 의사 결정 트리 모델 학습 진행
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 곡선 그리기
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 대각선 50% 선 그리기
plt.plot([0, 1], [0, 1], 'k--')
# 모델의 FPR 및 TPR 그리기
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

이제 등록된 환경을 검색한 다음 대체 학습 스크립트를 실행하는 새 실험에서 사용할 수 있습니다. 단, 의사 결정 트리 분류기에는 정규화 매개 변수가 필요하지 않으므로 이번에는 정규화 매개 변수가 없습니다.

In [ ]:
# 등록된 환경 가져오기
registered_env = Environment.get(ws, 'diabetes-experiment-env')

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='diabetes_training.py',
                              arguments = ['--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                              environment=registered_env,
                              docker_runtime_config=DockerConfiguration(use_docker=True)) # Use docker to host environment 

# 실험 제출
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

이번에는 실험이 더 빠르게 실행됩니다. 이전 실행에서 일치하는 환경이 캐시되었으므로 로컬 컴퓨팅에서 환경을 다시 만들 필요가 없기 때문입니다. 하지만 컴퓨팅 대상이 다르더라도 같은 환경이 작성되어 사용됩니다. 그러므로 실험 스크립트 실행 컨텍스트의 일관성이 유지됩니다.

이번에는 실험의 메트릭과 출력을 살펴보겠습니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 등록된 환경 확인

직접 만든 환경을 등록할 수도 있고, 일반 실험 유형에는 미리 작성된 "큐레이트" 환경을 활용할 수도 있습니다. 다음 코드는 등록된 모든 환경의 목록을 표시합니다.

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    print("Name",env)

모든 큐레이트 환경의 이름은 ***AzureML-***로 시작됩니다. 직접 만든 환경에는 이 접두사를 사용할 수 없습니다.

큐레이트 환경을 더 자세히 살펴보고 각 환경에 포함된 패키지를 확인해 보겠습니다.

In [ ]:
for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

## 컴퓨팅 클러스터 만들기

대부분의 경우 로컬 컴퓨팅 리소스만으로는 대량의 데이터를 처리해야 하는 복잡한 실험이나 장기 실행 실험을 처리하기 어려울 수 있습니다. 이러한 상황에서는 클라우드에서 컴퓨팅 리소스를 동적으로 만들고 사용하는 기능을 활용할 수 있습니다. Azure Machine Learning은 광범위한 컴퓨팅 대상을 지원합니다. 이러한 컴퓨팅 대상은 작업 영역에서 정의하고 실험을 실행하는 데 사용할 수 있으며, 리소스 사용 시에만 비용을 지불하면 됩니다.

컴퓨팅 클러스터는 [Azure Machine Learning Studio](https://ml.azure.com)에서 만들 수도 있고 Azure Machine Learning SDK를 사용하여 만들 수도 있습니다. 다음 코드 셀은 작업 영역에 지정된 이름의 컴퓨팅 클러스터가 있는지를 확인한 후 해당 클러스터가 없으면 만듭니다.

> **중요**: 컴퓨팅 클러스터를 실행하기 전에 아래 코드에서 *your-compute-cluster*를 실제 컴퓨팅 클러스터에 적합한 이름으로 변경합니다. 기존 클러스터가 있으면 해당 클러스터의 이름을 지정할 수 있습니다. 클러스터 이름은 2~16자 사이의 전역으로 고유한 이름이어야 합니다. 유효한 문자는 영문자, 숫자 및 - 문자입니다.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

## 원격 컴퓨팅 대상에서 실험 실행

이제 이전에 실행했던 실험을 다시 실행할 수 있습니다. 이번에는 방금 직접 만든 컴퓨팅 클러스터에서 실험을 실행합니다. 

> **참고**: 이번에는 실험을 실행하는 데 시간이 훨씬 오래 걸립니다. conda 환경이 포함된 이미지를 작성한 다음 클러스터 노드를 시작하고 이미지를 배포해야 스크립트를 실행할 수 있기 때문입니다. 따라서 당뇨병 학습 스크립트와 같은 간단한 실험에서는 효율적이지 않을 수도 있습니다. 하지만 몇 시간이 걸리는 훨씬 복잡한 실험을 실행해야 하는 경우 확장성이 더 높은 컴퓨팅을 동적으로 만들면 전체 실험 시간을 크게 줄일 수도 있습니다.

In [ ]:
# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=registered_env,
                                compute_target=cluster_name) 

# 실험 제출
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()

실험이 실행되는 동안 위의 위젯이나 [Azure Machine Learning Studio](https://ml.azure.com)에서 컴퓨팅 상태를 확인할 수 있습니다. 아래 코드를 사용하여 컴퓨팅 상태를 확인할 수도 있습니다.

In [ ]:
cluster_state = training_cluster.get_status()
print(cluster_state.allocation_state, cluster_state.current_node_count)

상태가 *steady*에서 *resizing*으로 바뀌려면 시간이 오래 걸리므로 잠시 휴식을 취하셔도 됩니다. 실행이 완료될 때까지 커널을 차단하려면 아래 셀을 실행합니다.

In [ ]:
run.wait_for_completion()

실험이 완료되고 나면 실험 실행에서 생성된 파일과 메트릭을 가져올 수 있습니다. 이번에는 이미지 작성 및 컴퓨팅 관리용 로그가 파일에 포함됩니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

이제 실험을 통해 학습된 모델을 등록할 수 있습니다.

In [ ]:
from azureml.core import Model

# 모델 등록
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Compute cluster'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 등록된 모델 목록 표시
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **자세한 정보**:
>
> - Azure Machine Learning의 환경에 대한 자세한 내용은 [Azure Machine Learning에서 소프트웨어 환경 만들기 및 사용](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments)을 참조하세요.
> - Azure Machine Learning의 컴퓨팅 대상에 대한 자세한 내용은 [Azure Machine Learning의 컴퓨팅 대상이란 무엇인가요?](https://docs.microsoft.com/azure/machine-learning/concept-compute-target)를 참조하세요.